In [ ]:
import sys
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from ydata_profiling import ProfileReport
import io
import tabulate
import helpers_eda_inicial
import helpers_cbc

In [ ]:
cbc, df_carreras = helpers_cbc.get_data('../../../assets/bronze/CBC/Sitacad_Tesis (1).xlsx')
actas = pd.read_csv('../../../assets/bronze/FCEN/FCEN_oficial_v2/reportes_actas.csv')
personas = pd.read_csv('../../../assets/bronze/FCEN/FCEN_oficial_v3/reporte_personas.csv')
cbc['Dni'] = cbc['Dni'].astype(str)
actas['fecha'] = pd.to_datetime(actas['fecha'], format='%Y-%m-%d')
actas['año'] = actas['fecha'].dt.year
actas['mes'] = actas['fecha'].dt.month

Vamos a trabajar con las personas de LU 2021 y 2022

In [ ]:
personas_2021_2022 = personas[(personas['año_inscripcion_facultad'] == 2021) | (personas['año_inscripcion_facultad'] == 2022)]
personas_2021_2022['año_inscripcion_facultad'].value_counts().sort_index()

In [72]:
personas_2021_2022_dnis = personas_2021_2022['dni'].unique()

Nos guardamos las actas de las personas con LU 2021 y 2022

In [73]:
actas_inscriptos_2021_2022 = actas[actas['dni'].isin(personas_2021_2022_dnis)]

In [74]:
personas_con_actas = personas_2021_2022.merge(actas_inscriptos_2021_2022, on='dni', how='left')

In [ ]:
personas_con_actas.groupby('año_inscripcion_facultad')['dni'].nunique().sort_index()

In [ ]:
personas_con_actas.columns

Vemos que hay personas que no tienen actas asignadas, pues no tienen un valor en la columna 'resultado' (donde antes siempre teníamos los valores Aprobado, Reprobado o Ausente) Pero a nosotros nos interesa quedarnos solamente con aquellas personas que tienen regitros en siu-guarani.

In [ ]:
personas_con_actas['resultado'].value_counts(dropna = False)

In [78]:
alumnos_sin_actas = personas_con_actas[personas_con_actas['resultado'].isna()]['dni'].unique()

In [ ]:
personas_con_actas = personas_con_actas[~personas_con_actas['resultado'].isna()]
personas_con_actas[personas_con_actas['dni'].isin(alumnos_sin_actas)]

In [ ]:
personas_con_actas['resultado'].value_counts(dropna = False)

In [ ]:
personas_con_actas.groupby('año_inscripcion_facultad')['dni'].nunique().sort_index()

Veo además que hay personas con actas anteriores a 2021, lo cual es extraño, pero podrían ser datos mal ingresados o tal vez está relacionado con la pandemia, no se si será gente que no pudo anotarse durante el 2020 pero les permitieron cursar o algo así.

In [ ]:
actas_inscriptos_2021_2022['año'].value_counts().sort_index()

In [ ]:
actas_inscriptos_2021_2022[actas_inscriptos_2021_2022['año'] < 2021]['dni'].unique().shape[0]

In [ ]:
actas_inscriptos_2021_2022[actas_inscriptos_2021_2022['año'] == 2020]['dni'].unique().shape[0]

In [ ]:
actas_inscriptos_2021_2022[actas_inscriptos_2021_2022['año'] == 2020]['materia'].value_counts().sort_index()

In [ ]:
personas_con_actas.groupby('año_inscripcion_facultad')['año'].value_counts().sort_index()

In [ ]:
personas_con_actas.groupby(['año_inscripcion_facultad','año'])['dni'].nunique().sort_index()

Me fijo de eliminar las actas anteriores al año de inscripción de la personas. Al sacar las actas anteriores a su año de inscripción me quedan 3083 de las 3170 que eran originalmente. Sin embargo, veo que hay muchas personas con LU 2021 que tienen registros de actas en 2020, por lo que me voy a fijar de directamente sacar las personas que tienen registros en 2020.

In [88]:
personas_con_actas_2021_o_mas = personas_con_actas[personas_con_actas['año_inscripcion_facultad'] <= personas_con_actas['año']]

In [ ]:
personas_con_actas_2021_o_mas['dni'].nunique(), personas_con_actas['dni'].nunique()

In [ ]:
personas_con_actas.groupby('año_inscripcion_facultad')['dni'].nunique().sort_index()

Me fijo de directamente eliminar a las personas que tienen actas anteriores a su año de inscripción.

In [91]:
dnis_actas_anteriores_inscripcion = personas_con_actas[personas_con_actas['año_inscripcion_facultad'] > personas_con_actas['año']]['dni'].unique()

In [ ]:
dnis_actas_anteriores_inscripcion.size, personas_con_actas['dni'].nunique()

In [93]:
personas_con_actas_restringido_2021 = personas_con_actas[~personas_con_actas['dni'].isin(dnis_actas_anteriores_inscripcion)]

In [ ]:
personas_con_actas_restringido_2021['dni'].nunique(), personas_con_actas['dni'].nunique()

In [ ]:
personas_con_actas_restringido_2021.groupby(['año_inscripcion_facultad','año'])['dni'].nunique().sort_index()

In [ ]:
personas_con_actas_restringido_2021.groupby('año_inscripcion_facultad')['dni'].nunique().sort_index()

## Con lo que vamos a trabajar

Entonces, trabajamos con personas_con_actas_restringido_2021, este dataset contiene los datos personales y las actas de la FCEN. Lo comenzamos a analizar, pero luego debemos fijarnos de juntarlo con los datos del CBC y encuestas de manera correcta.

#### No importa si no tenemos el historial de su CBC

In [ ]:
personas_con_actas_restringido_2021.dtypes

In [ ]:
personas_con_actas_restringido_2021.drop(columns=['nivel_estudio','estado_nivel_estudio'], inplace=True)

In [99]:
#helpers_eda_inicial.dataset_profiling(personas_con_actas_restringido_2021, 'personas_con_actas_restringido_2021')

### Deberíamos asignar semestre relativo

Pues de esta manera nos aseguramos de trabajar con gente que tenga los registros necesarios (aunque ya descartamos a la gente que no tenía registros o cuyos registros eran anteriores a su año de inscripción)

Cuál variable estamos tomando como target? El alumno se encuentra en riesgo de abandono en el semestre relativo 4, donde riesgo de abandono sería #inscripciones + #tps_aprobados + #finales < 3

Por lo tanto:

Toma valor 0 si
#inscripciones + #tps_aprobados + #finales >= 3 en el semestre relativo 4

Toma valor 1 si 
#inscripciones + #tps_aprobados + #finales < 3 en el semestre relativo 4
Es decir, el alumno se encuentra en riesgo de abandono en el semestre relativo 4

In [ ]:
personas_con_actas_restringido_2021.columns

Primero lo defino como si todos estuvieran desde el primer cuatrimestre de cada año

In [ ]:
conditions_iniciales = [
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022) & (personas_con_actas_restringido_2021['año'] == 2022) & (personas_con_actas_restringido_2021['mes'].between(1, 8)),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022) & (personas_con_actas_restringido_2021['año'] == 2022) & (personas_con_actas_restringido_2021['mes'].between(9, 12)),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022) & (personas_con_actas_restringido_2021['año'] == 2023) & (personas_con_actas_restringido_2021['mes'].isin([1, 2])),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022) & (personas_con_actas_restringido_2021['año'] == 2023) & (personas_con_actas_restringido_2021['mes'].between(3, 8)),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022) & (personas_con_actas_restringido_2021['año'] == 2023) & (personas_con_actas_restringido_2021['mes'].between(9, 12)),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022) & (personas_con_actas_restringido_2021['año'] == 2024) & (personas_con_actas_restringido_2021['mes'].isin([1, 2])),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022) & (personas_con_actas_restringido_2021['año'] == 2024) & (personas_con_actas_restringido_2021['mes'].between(3, 8)),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022) & (personas_con_actas_restringido_2021['año'] == 2024) & (personas_con_actas_restringido_2021['mes'].between(9, 12)),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022) & (personas_con_actas_restringido_2021['año'] == 2025), ##Ahora aclaro para 2021
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2021) & (personas_con_actas_restringido_2021['año'] == 2021) & (personas_con_actas_restringido_2021['mes'].between(1, 8)),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2021) & (personas_con_actas_restringido_2021['año'] == 2021) & (personas_con_actas_restringido_2021['mes'].between(9, 12)),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2021) & (personas_con_actas_restringido_2021['año'] == 2022) & (personas_con_actas_restringido_2021['mes'].isin([1, 2])),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2021) & (personas_con_actas_restringido_2021['año'] == 2022) & (personas_con_actas_restringido_2021['mes'].between(3, 8)),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2021) & (personas_con_actas_restringido_2021['año'] == 2022) & (personas_con_actas_restringido_2021['mes'].between(9, 12)),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2021) & (personas_con_actas_restringido_2021['año'] == 2023) & (personas_con_actas_restringido_2021['mes'].isin([1, 2])),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2021) & (personas_con_actas_restringido_2021['año'] == 2023) & (personas_con_actas_restringido_2021['mes'].between(3, 8)),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2021) & (personas_con_actas_restringido_2021['año'] == 2023) & (personas_con_actas_restringido_2021['mes'].between(9, 12)),
    (personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2021) & (personas_con_actas_restringido_2021['año'] == 2024) & (personas_con_actas_restringido_2021['mes'].isin([1, 2])),
]
values = [0, 1, 1, 2, 3, 3, 4, 5, 5, 0, 1, 1, 2, 3, 3, 4, 5, 5]

personas_con_actas_restringido_2021['semestre_relativo'] = np.select(conditions_iniciales, values, default=np.nan)

Vemos que hay un gran conjunto de actas que no se les asignó un semestre relativo. Sin embargo, la gran mayoría de esas actas pertenecen a personas que tienen asignadas actas para los semestres relativos 0 y 1

In [ ]:
personas_con_actas_restringido_2021['semestre_relativo'].value_counts(dropna = False)

In [ ]:
actas_sin_semestre_relativo = personas_con_actas_restringido_2021[personas_con_actas_restringido_2021['semestre_relativo'].isna()]['dni'].unique()
actas_sin_semestre_relativo.shape

In [ ]:
personas_con_actas_restringido_2021[personas_con_actas_restringido_2021['dni'].isin(actas_sin_semestre_relativo) & (personas_con_actas_restringido_2021['semestre_relativo'].isin([0,1]))]['dni'].nunique()

Por ahora, voy a pasar a asignar si la persona es del primer o segundo semestre de cada año.

In [ ]:
cohorte_20221c = personas_con_actas_restringido_2021[(personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022) & (personas_con_actas_restringido_2021['semestre_relativo'] == 0)]['dni'].unique()
personas_con_actas_restringido_2021['es_cohorte_2022C1'] = personas_con_actas_restringido_2021['dni'].isin(cohorte_20221c)

In [ ]:
cohorte_20221c.size

El resto de 2022 debería ser de la cohorte 2022C2, por lo que nos quedamos con ellos y les restamos 1 a su valor en la columna 'semestre_relativo'. Luego, los que sigan sin tener un semestre 0, los sacamos.

In [ ]:
cohorte_20222c = personas_con_actas_restringido_2021[(personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022) & (~personas_con_actas_restringido_2021['dni'].isin(cohorte_20221c))]['dni'].unique()
personas_con_actas_restringido_2021['es_cohorte_2022C2']= personas_con_actas_restringido_2021['dni'].isin(cohorte_20222c)
cohorte_20222c.size

In [108]:
personas_con_actas_restringido_2021.loc[personas_con_actas_restringido_2021['es_cohorte_2022C2'], 'semestre_relativo'] -= 1

In [ ]:
# 1. Filtrar los DNI que *sí* tienen al menos una fila con semestre_relativo_20222c == 0
dnis_con_semestre_0 = personas_con_actas_restringido_2021.loc[
    (personas_con_actas_restringido_2021['es_cohorte_2022C2']) & (personas_con_actas_restringido_2021['semestre_relativo'] == 0), 'dni'
].unique()

# 2. Filtrar los DNI que *no* están en esa lista
dnis_sin_semestre_0 = personas_con_actas_restringido_2021.loc[
   (personas_con_actas_restringido_2021['es_cohorte_2022C2']) & (~personas_con_actas_restringido_2021['dni'].isin(dnis_con_semestre_0)), 'dni'
].unique()

# 3. Mostrar los resultados
print(dnis_sin_semestre_0)

In [110]:
personas_con_actas_restringido_2021 = personas_con_actas_restringido_2021[~personas_con_actas_restringido_2021['dni'].isin(dnis_sin_semestre_0)]

Hasta este punto debería haber quedado completamente arreglado todos los que se inscribieron en 2022

In [ ]:
cohorte_20222c = personas_con_actas_restringido_2021[(personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022) & (personas_con_actas_restringido_2021['es_cohorte_2022C2'])]['dni'].unique()
cohorte_20222c.size

In [ ]:
personas_con_actas_restringido_2021[personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2022]['dni'].nunique()

Ahora debemos arreglar la cohorte de 2021.

In [ ]:
cohorte_20211c = personas_con_actas_restringido_2021[(personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2021) & (personas_con_actas_restringido_2021['semestre_relativo'] == 0)]['dni'].unique()
personas_con_actas_restringido_2021['es_cohorte_2021C1'] = personas_con_actas_restringido_2021['dni'].isin(cohorte_20211c)
cohorte_20211c.size

In [ ]:
cohorte_20212c = personas_con_actas_restringido_2021[(personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2021) & (~personas_con_actas_restringido_2021['dni'].isin(cohorte_20211c))]['dni'].unique()
personas_con_actas_restringido_2021['es_cohorte_2021C2']= personas_con_actas_restringido_2021['dni'].isin(cohorte_20212c)
cohorte_20212c.size

In [115]:
personas_con_actas_restringido_2021.loc[personas_con_actas_restringido_2021['es_cohorte_2021C2'], 'semestre_relativo'] -= 1

In [ ]:
# 1. Filtrar los DNI que *sí* tienen al menos una fila con semestre_relativo_20222c == 0
dnis_con_semestre_0 = personas_con_actas_restringido_2021.loc[
    (personas_con_actas_restringido_2021['es_cohorte_2021C2']) & (personas_con_actas_restringido_2021['semestre_relativo'] == 0), 'dni'
].unique()

# 2. Filtrar los DNI que *no* están en esa lista
dnis_sin_semestre_0 = personas_con_actas_restringido_2021.loc[
   (personas_con_actas_restringido_2021['es_cohorte_2021C2']) & (~personas_con_actas_restringido_2021['dni'].isin(dnis_con_semestre_0)), 'dni'
].unique()

# 3. Mostrar los resultados
print(dnis_sin_semestre_0)

In [ ]:
personas_con_actas_restringido_2021 = personas_con_actas_restringido_2021[~personas_con_actas_restringido_2021['dni'].isin(dnis_sin_semestre_0)]
cohorte_20212c = personas_con_actas_restringido_2021[(personas_con_actas_restringido_2021['año_inscripcion_facultad'] == 2021) & (personas_con_actas_restringido_2021['es_cohorte_2021C2'])]['dni'].unique()
cohorte_20212c.size

In [ ]:
personas_con_actas_restringido_2021['semestre_relativo'].value_counts(dropna = False)

In [ ]:
personas_con_actas_restringido_2021[personas_con_actas_restringido_2021['semestre_relativo'].isnull()]['fecha'].hist()

In [ ]:
personas_con_actas_restringido_2021 = personas_con_actas_restringido_2021[personas_con_actas_restringido_2021['semestre_relativo'] < 5]
personas_con_actas_restringido_2021['semestre_relativo'].value_counts(dropna = False)

In [ ]:
personas_con_actas_restringido_2021['dni'].nunique()

Me quedo con el dataset que tenga solamente las personas para las que encontré actas y que cumplen con el requisito de que sus actas sean posteriores a su año de inscripción.

In [ ]:
dni_personas_con_actas_restringido_2021 = personas_con_actas_restringido_2021['dni'].unique()
personas_restringido_2021_a_2022 = personas[personas['dni'].isin(dni_personas_con_actas_restringido_2021)]
personas_restringido_2021_a_2022['dni'].nunique(), personas_con_actas_restringido_2021['dni'].nunique()

In [ ]:
personas_restringido_2021_a_2022.drop(columns=['nivel_estudio','estado_nivel_estudio'], inplace=True)

In [124]:
#helpers_eda_inicial.dataset_profiling(personas_restringido_2021_a_2022, 'personas_restringido_2021_a_2022')

### Edad

Recordemos que para el estudio de características de las personas usamos personas_restringido_2021_a_2022, que usa los DNI de personas_con_actas_restringido_2021

In [ ]:
# hago un barplot de personas_restringido_2021_a_2022['edad']
%matplotlib inline

plt.figure(figsize=(10, 5))
sns.histplot(personas_restringido_2021_a_2022['edad'], bins=20)
plt.title('Distribución de edades de los alumnos')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# hago un boxplot de personas_restringido_2021_a_2022['edad']
plt.figure(figsize=(10, 5))
sns.boxplot(x=personas_restringido_2021_a_2022['edad'])
plt.title('Boxplot de edades de los alumnos')
plt.xlabel('Edad')
plt.show()

In [ ]:
# reemplazo por NA las edades iguales a 125
personas_restringido_2021_a_2022['edad'].replace(125, np.nan, inplace=True)
personas_con_actas_restringido_2021['edad'].replace(125, np.nan, inplace=True)

In [ ]:
# hago un barplot de personas_restringido_2021_a_2022['edad']
plt.figure(figsize=(10, 5))
sns.histplot(personas_restringido_2021_a_2022['edad'], bins=20)
plt.title('Distribución de edades de los alumnos en el año 2025')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# si en 'año_inscripcion_facultad' es 2021, entonces 'edad' es igual a 'edad' - 4. Pero si en 'año_inscripcion_facultad' es 2022, entonces 'edad' es igual a 'edad' - 3
personas_restringido_2021_a_2022['edad_inscripcion'] = np.where(personas_restringido_2021_a_2022['año_inscripcion_facultad'] == 2021,
                                                      personas_restringido_2021_a_2022['edad'] - 4,
                                                      personas_restringido_2021_a_2022['edad'] - 3)
# hago un barplot de personas_restringido_2021_a_2022['edad_inscripcion']
plt.figure(figsize=(10, 5))
sns.histplot(personas_restringido_2021_a_2022['edad_inscripcion'], bins=30, stat='probability')
plt.title('Distribución de edades de los alumnos al inscribirse', fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Edad', fontsize=15)
plt.ylabel('Proporción', fontsize=15)
plt.show()

In [ ]:
# hago un boxplot de personas_restringido_2021_a_2022['edad']
plt.figure(figsize=(10, 5))
sns.boxplot(x=personas_restringido_2021_a_2022['edad_inscripcion'])
plt.title('Boxplot de edades de los alumnos al inscribirse')
plt.xlabel('Edad')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(personas_restringido_2021_a_2022[~personas_restringido_2021_a_2022['edad_inscripcion'].isna()]['edad_inscripcion'], bins=30, stat='probability')
plt.gca().yaxis  # eje Y en %
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(5))  # ticks cada 1 año

plt.title('Distribución de edades de los alumnos al inscribirse', fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Edad', fontsize=12)
plt.ylabel('Porcentaje', fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(personas_restringido_2021_a_2022[(~personas_restringido_2021_a_2022['edad_inscripcion'].isna()) & (personas_restringido_2021_a_2022['año_inscripcion_facultad'] == 2021)]['edad_inscripcion'], bins=30, stat='probability')
plt.gca().yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))  # eje Y en %
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(5))  # ticks cada 1 año

plt.title('Distribución de edades de los alumnos al inscribirse en 2021')
plt.xlabel('Edad')
plt.ylabel('Porcentaje')
plt.show()

In [ ]:
# hago un boxplot de personas_restringido_2021_a_2022['edad']
plt.figure(figsize=(10, 5))
sns.boxplot(x=personas_restringido_2021_a_2022[(~personas_restringido_2021_a_2022['edad_inscripcion'].isna()) & (personas_restringido_2021_a_2022['año_inscripcion_facultad'] == 2021)]['edad_inscripcion'])
plt.title('Boxplot de edades de los alumnos al inscribirse en 2021')
plt.xlabel('Edad')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(personas_restringido_2021_a_2022[(~personas_restringido_2021_a_2022['edad_inscripcion'].isna()) & (personas_restringido_2021_a_2022['año_inscripcion_facultad'] == 2022)]['edad_inscripcion'], bins=30, stat='probability')
plt.gca().yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))  # eje Y en %
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(5))  # ticks cada 1 año

plt.title('Distribución de edades de los alumnos al inscribirse en 2022')
plt.xlabel('Edad')
plt.ylabel('Porcentaje')
plt.show()

In [ ]:
# hago un boxplot de personas_restringido_2021_a_2022['edad']
plt.figure(figsize=(10, 5))
sns.boxplot(x=personas_restringido_2021_a_2022[(~personas_restringido_2021_a_2022['edad_inscripcion'].isna()) & (personas_restringido_2021_a_2022['año_inscripcion_facultad'] == 2022)]['edad_inscripcion'])
plt.title('Boxplot de edades de los alumnos al inscribirse en 2022')
plt.xlabel('Edad')
plt.show()

### Sexo

In [ ]:
# barplot de la variable sexo del dataset personas_restringido_2021_a_2022
plt.figure(figsize=(10, 5))
sns.countplot(x='sexo', data=personas_restringido_2021_a_2022)
plt.title('Distribución de la variable sexo', fontsize=15)
plt.xticks(fontsize=13)
plt.yticks(fontsize=12)
plt.xlabel('Sexo', fontsize=15)
plt.ylabel('Cantidad', fontsize=15)
plt.show()

In [ ]:
# barplot de la variable sexo del dataset personas_restringido_2021_a_2022
plt.figure(figsize=(10, 5))
sns.countplot(x='sexo', data=personas_restringido_2021_a_2022, stat='probability')
plt.title('Distribución de la variable sexo', fontsize=15)
plt.xticks(fontsize=13)
plt.yticks(fontsize=12)
plt.xlabel('Sexo', fontsize=15)
plt.ylabel('Proporción', fontsize=15)
plt.show()

In [ ]:
personas_restringido_2021_a_2022['sexo'].value_counts(normalize=True)

In [ ]:
# barplot de la variable sexo del dataset personas_restringido_2021_a_2022, agrupado por año_inscripcion_facultad
plt.figure(figsize=(10, 5))
sns.countplot(x='sexo', data=personas_restringido_2021_a_2022, hue='año_inscripcion_facultad', stat='probability', palette='muted')
plt.title('Distribución de sexo de los alumnos')
plt.xlabel('Sexo')
plt.ylabel('Frecuencia')
plt.legend(title='Año de inscripción')
plt.show()

In [ ]:
# barplot de la variable sexo del dataset personas_restringido_2021_a_2022
plt.figure(figsize=(10, 5))
sns.countplot(x='sexo', data=personas_restringido_2021_a_2022[personas_restringido_2021_a_2022['año_inscripcion_facultad'] == 2021], stat='probability')
plt.title('Distribución de sexo de los alumnos con inscripción en 2021')
plt.xlabel('Sexo')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# barplot de la variable sexo del dataset personas_restringido_2021_a_2022
plt.figure(figsize=(10, 5))
sns.countplot(x='sexo', data=personas_restringido_2021_a_2022[personas_restringido_2021_a_2022['año_inscripcion_facultad'] == 2022], stat='probability', order=['Masculino', 'Femenino'])
plt.title('Distribución de sexo de los alumnos con inscripción en 2022')
plt.xlabel('Sexo')
plt.ylabel('Frecuencia')
plt.show()

### Nacionalidad

In [ ]:
# barplot de la variable sexo del dataset personas_restringido_2021_a_2022
plt.figure(figsize=(10, 5))
sns.countplot(x='nacionalidad', data=personas_restringido_2021_a_2022, stat='probability')
plt.title('Distribución de la nacionalidad de los alumnos')
plt.xlabel('Nacionalidad')
plt.ylabel('Frecuencia')
plt.show()

### Variables que se usan para calcular tiempo de viaje

In [ ]:
personas_restringido_2021_a_2022['pais'].value_counts(dropna = False).sort_index()

In [ ]:
personas_restringido_2021_a_2022[personas_restringido_2021_a_2022['pais'].isna()]

In [ ]:
personas_restringido_2021_a_2022[personas_restringido_2021_a_2022['codigo_postal'].isna()]['pais'].value_counts(dropna = False)

In [ ]:
personas_restringido_2021_a_2022[personas_restringido_2021_a_2022['localidad'].isna()]['pais'].value_counts(dropna = False)

In [ ]:
personas_restringido_2021_a_2022[personas_restringido_2021_a_2022['departamento'].isna()]['pais'].value_counts(dropna = False)

In [ ]:
personas_restringido_2021_a_2022[personas_restringido_2021_a_2022['provincia'].isna()]['pais'].value_counts(dropna = False)

### dominio_mail

In [ ]:
# barplot de la variable sexo del dataset personas_restringido_2021_a_2022
plt.figure(figsize=(10, 5))
sns.countplot(x='dominio_email', data=personas_restringido_2021_a_2022, stat='probability')
plt.title('Distribución del dominio_email de los alumnos')
plt.xticks(rotation=45, ha='right') 
plt.xlabel('Dominio Email')
plt.ylabel('Frecuencia')
plt.show()

### Situación madre y padre

In [ ]:
# Función para preparar los datos con proporciones y NaN como categoría
def preparar_datos(columna):
    serie = personas_restringido_2021_a_2022[columna].fillna('Faltante')  # incluir NaN como "Faltante"
    proporciones = serie.value_counts(normalize=True).reset_index()
    proporciones.columns = ['categoria', 'proporcion']
    return proporciones

# Preparar los datos para cada columna
df_padre = preparar_datos('situacion_padre')
df_madre = preparar_datos('situacion_madre')

# Crear figura con subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# Barplot para situacion_padre
sns.barplot(data=df_padre, x='categoria', y='proporcion', ax=axes[0])
axes[0].set_title('Situación del Padre')
axes[0].set_xlabel('Situación')
axes[0].set_ylabel('Proporción')
axes[0].tick_params(axis='x', rotation=45)

# Barplot para situacion_madre
sns.barplot(data=df_madre, x='categoria', y='proporcion', ax=axes[1])
axes[1].set_title('Situación de la Madre')
axes[1].set_xlabel('Situación')
axes[1].set_ylabel('')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Preparar los datos para cada columna
df_padre = preparar_datos('nivel_estudio_padre')
df_madre = preparar_datos('nivel_estudio_madre')

# Crear figura con subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# Barplot para situacion_padre
sns.barplot(data=df_padre, x='categoria', y='proporcion', ax=axes[0])
axes[0].set_title('Nivel de estudio Padre', fontsize=15)
axes[0].set_xlabel('Nivel de estudio', fontsize=14)
axes[0].set_ylabel('Proporción', fontsize=14)
axes[0].tick_params(axis='x', rotation=90)

# Barplot para situacion_madre
sns.barplot(data=df_madre, x='categoria', y='proporcion', ax=axes[1])
axes[1].set_title('Nivel de Estudio Madre', fontsize=15)
axes[1].set_xlabel('Nivel de estudio', fontsize=14)
axes[1].set_ylabel('', fontsize=14)
axes[1].tick_params(axis='x', rotation=90)

plt.tight_layout()
plt.show()

In [ ]:
df_padre

In [ ]:
df_madre

In [ ]:
# Función para preparar los datos con conteo y NaN como categoría
def preparar_datos_conteo(columna):
    serie = personas_restringido_2021_a_2022[columna].fillna('Faltante')  # incluir NaN como "Faltante"
    conteos = serie.value_counts().reset_index()
    conteos.columns = ['categoria', 'cantidad']
    return conteos

# Preparar los datos para cada columna
df_padre = preparar_datos_conteo('nivel_estudio_padre')
df_madre = preparar_datos_conteo('nivel_estudio_madre')

# Crear figura con subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# Barplot para nivel_estudio_padre
sns.barplot(data=df_padre, x='categoria', y='cantidad', ax=axes[0])
axes[0].set_title('Nivel de estudio Padre')
axes[0].set_xlabel('Nivel de estudio')
axes[0].set_ylabel('Cantidad')
axes[0].tick_params(axis='x', rotation=90)

# Barplot para nivel_estudio_madre
sns.barplot(data=df_madre, x='categoria', y='cantidad', ax=axes[1])
axes[1].set_title('Nivel de Estudio Madre')
axes[1].set_xlabel('Nivel de estudio')
axes[1].set_ylabel('')
axes[1].tick_params(axis='x', rotation=90)

plt.tight_layout()
plt.show()


In [ ]:
helpers_eda_inicial.display_missing_values(personas_restringido_2021_a_2022)

### acceso_a_internet

In [ ]:
# Crear una copia de la columna, reemplazando NaN por "Faltante"
df = personas_restringido_2021_a_2022[['dni', 'acceso_a_internet']].copy()
#df['acceso_a_internet'] = df['acceso_a_internet'].fillna('Faltante')

# Crear el barplot con proporciones
plt.figure(figsize=(10, 5))
sns.countplot(x='acceso_a_internet', data=df, stat='probability')
plt.title('Distribución del acceso a internet de los alumnos')
plt.xticks(rotation=45, ha='right') 
plt.xlabel('Acceso a internet')
plt.ylabel('Proporción')
plt.tight_layout()
plt.show()

### situacion_laboral

In [ ]:
# Crear una copia de la columna, reemplazando NaN por "Faltante"
nombres_largos = {
    'No trabajó y no buscó trabajo (no esta pensando en trabajar)': 'No trabaja ni busca trabajo',
    'Trabajó al menos una hora (incluye a los que no trabajaron por licencia, vacaciones, enfermedad)': 'Trabajó al menos una hora',
    'No trabajó y buscó trabajo en algún momento de los últimos 30 días': 'No trabaja pero busca',
    'Desconoce': 'Desconoce'
}

df = personas_restringido_2021_a_2022[['dni', 'situacion_laboral']].copy()
df['situacion_laboral'] = df['situacion_laboral'].replace(nombres_largos)
df['situacion_laboral'] = df['situacion_laboral'].fillna('Faltante')

# Crear el barplot con proporciones
plt.figure(figsize=(11, 6))
sns.countplot(x='situacion_laboral', data=df, order=df['situacion_laboral'].value_counts(dropna = False).index)
plt.title('Distribución de la situación laboral', fontsize=15)
plt.xticks(rotation=45, ha='right', fontsize=12) 
plt.yticks(fontsize=12)
plt.xlabel('Situación laboral', fontsize=15)
plt.ylabel('Cantidad', fontsize=15)

plt.tight_layout()
plt.show()

In [ ]:
# Crear una copia de la columna, reemplazando NaN por "Faltante"
nombres_largos = {
    'No trabajó y no buscó trabajo (no esta pensando en trabajar)': 'No trabaja ni busca trabajo',
    'Trabajó al menos una hora (incluye a los que no trabajaron por licencia, vacaciones, enfermedad)': 'Trabajó al menos una hora',
    'No trabajó y buscó trabajo en algún momento de los últimos 30 días': 'No trabaja pero busca',
    'Desconoce': 'Desconoce'
}

df = personas_restringido_2021_a_2022[['dni', 'situacion_laboral']].copy()
df['situacion_laboral'] = df['situacion_laboral'].replace(nombres_largos)
df['situacion_laboral'] = df['situacion_laboral'].fillna('Faltante')

# Crear el barplot con proporciones
plt.figure(figsize=(11, 6))
sns.countplot(x='situacion_laboral', data=df, stat='probability', order=df['situacion_laboral'].value_counts(dropna = False).index)
plt.title('Distribución de la situación laboral', fontsize=15)
plt.xticks(rotation=45, ha='right', fontsize=12) 
plt.yticks(fontsize=12)
plt.xlabel('Situación laboral', fontsize=15)
plt.ylabel('Proporción', fontsize=15)

plt.tight_layout()
plt.show()

In [ ]:
df['situacion_laboral'].value_counts(normalize=True)

### año_inscripcion_facultad

In [ ]:
df = personas_restringido_2021_a_2022[['dni', 'año_inscripcion_facultad']].copy()
df['año_inscripcion_facultad'] = df['año_inscripcion_facultad'].fillna('Faltante')

# Crear el barplot con proporciones
plt.figure(figsize=(10, 5))
sns.countplot(x='año_inscripcion_facultad', data=df, stat='probability', order=df['año_inscripcion_facultad'].value_counts(dropna = False).index)
plt.title('Distribución del año de inscripcion a la facultad')
plt.xticks(rotation=45, ha='right') 
plt.xlabel('Año de inscripcion a la facultad')
plt.ylabel('Proporción')
plt.tight_layout()
plt.show()

## Ahora quiero ver por semestre asignado

In [ ]:
#personas_restringido_2021_a_2022.to_csv('../../../../assets/silver/data_utilizada/solo_datos_personales.csv')

In [ ]:
#personas_con_actas_restringido_2021.to_csv('../../../../assets/silver/data_utilizada/personas_un_acta_por_fila.csv')

In [ ]:
semestre_de_inscripcion = personas_con_actas_restringido_2021[
    ['dni', 'es_cohorte_2022C1', 'es_cohorte_2022C2', 'es_cohorte_2021C1', 'es_cohorte_2021C2']
].drop_duplicates()
semestre_de_inscripcion['dni'].nunique()

In [158]:
# Contar cuántos True hay por fila
cohorte_cols = ['es_cohorte_2022C1', 'es_cohorte_2022C2', 'es_cohorte_2021C1', 'es_cohorte_2021C2']
semestre_de_inscripcion['cohortes_true_count'] = semestre_de_inscripcion[cohorte_cols].sum(axis=1)

# Filtrar los casos incorrectos (donde no hay exactamente 1 True)
casos_invalidos = semestre_de_inscripcion[semestre_de_inscripcion['cohortes_true_count'] != 1]

In [ ]:
if casos_invalidos.empty:
    print("✔️ Todos los DNI tienen exactamente una cohorte asignada.")
else:
    print("❌ Hay DNIs con múltiples o ninguna cohorte asignada:")
    print(casos_invalidos)

In [ ]:
cohorte_cols = ['es_cohorte_2022C1', 'es_cohorte_2022C2', 'es_cohorte_2021C1', 'es_cohorte_2021C2']
semestre_de_inscripcion['cohorte'] = semestre_de_inscripcion[cohorte_cols].idxmax(axis=1)
semestre_de_inscripcion['cohorte'] = semestre_de_inscripcion['cohorte'].str.replace('es_cohorte_', '')

sns.countplot(data=semestre_de_inscripcion, x='cohorte', order=sorted(semestre_de_inscripcion['cohorte'].unique()))
plt.title("Distribución por cohorte", fontsize=15)
plt.xlabel("Cohorte", fontsize=15)
plt.ylabel("Cantidad de personas", fontsize=15)
plt.xticks(rotation=0, fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Calcular proporciones
cohorte_props = semestre_de_inscripcion['cohorte'].value_counts(normalize=True).reset_index()
cohorte_props.columns = ['cohorte', 'proporcion']

# Ordenar por cohorte si querés que aparezcan en orden cronológico
cohorte_props = cohorte_props.sort_values('cohorte')

# Graficar
sns.barplot(data=cohorte_props, x='cohorte', y='proporcion')
plt.title("Distribución por cohorte", fontsize=15)
plt.xlabel("Cohorte", fontsize=15)
plt.ylabel("Proporción", fontsize=15)
plt.xticks(rotation=0, fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
cohorte_props

In [ ]:
personas_con_actas_restringido_2021['carrera'].isna().sum()

In [ ]:
# Paso 1: Contar las veces que cada persona se inscribió a cada carrera
conteos = personas_con_actas_restringido_2021.groupby(['dni', 'carrera']).size().reset_index(name='conteo')

# Paso 2: Para cada dni, quedarte con la carrera de mayor conteo
conteos = conteos.sort_values(['dni', 'conteo'], ascending=[True, False])
carrera_mas_frecuente = conteos.drop_duplicates(subset='dni', keep='first')
carrera_mas_frecuente

In [ ]:
# Ver distribución como tabla
distribucion = carrera_mas_frecuente['carrera'].value_counts(dropna= False)

print(distribucion)

# Gráfico de barras
distribucion.plot(kind='bar', figsize=(15, 5))
plt.title('Distribución de carreras más frecuentes por persona', fontsize=15)
plt.xlabel('Carrera Principal', fontsize=12)
plt.ylabel('Cantidad de personas', fontsize=12)
plt.xticks(rotation=90, fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
proporcion = carrera_mas_frecuente['carrera'].value_counts(normalize=True)
print(proporcion)

proporcion.plot(kind='bar', figsize=(15, 5))
plt.title('Proporción de carreras más frecuentes por persona')
plt.xlabel('Carrera')
plt.ylabel('Proporción')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Calcular distribución
distribucion = carrera_mas_frecuente['carrera'].value_counts(normalize=True)

# Agrupar carreras con 63 o menos casos
umbral = 0.017
carreras_comunes = distribucion[distribucion > umbral]
otros_total = distribucion[distribucion <= umbral].sum()

# Crear nueva serie combinada
distribucion_modificada = carreras_comunes.copy()
distribucion_modificada['Otras carreras'] = otros_total

# Mapeo de nombres personalizados
etiquetas_personalizadas = {
    'Licenciatura en Ciencias de la Computación': 'Computación',
    'Licenciatura en Ciencias Biológicas': 'Biología',
    'Licenciatura en Ciencias Físicas': 'Física',
    'Licenciatura en Ciencias de Datos': 'Datos',
    'Licenciatura en Ciencias Químicas': 'Química',
    'Licenciatura en Ciencias Matemáticas': 'Matemática',
    'Licenciatura en Ciencias de la Atmósfera': 'Atmósfera',
    'Licenciatura en Ciencia y Tecnología de Alimentos': 'Alimentos',
    'Licenciatura en Ciencias Geológicas': 'Geología',
    'Otras carreras': 'Otras carreras'
}

# Renombrar los índices según el diccionario
distribucion_modificada.index = distribucion_modificada.index.map(lambda x: etiquetas_personalizadas.get(x, x))

# Ordenar por cantidad descendente
distribucion_modificada = distribucion_modificada.sort_values(ascending=False)

# Graficar
distribucion_modificada.plot(kind='bar', figsize=(9, 5), color='royalblue')
plt.title('Distribución de carrera principal', fontsize=15)
plt.xlabel('Carrera Principal', fontsize=12)
plt.ylabel('Proporción', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
# Calcular distribución
distribucion = carrera_mas_frecuente['carrera'].value_counts(dropna=False)

# Agrupar carreras con 63 o menos casos
umbral = 51
carreras_comunes = distribucion[distribucion > umbral]
otros_total = distribucion[distribucion <= umbral].sum()

# Crear nueva serie combinada
distribucion_modificada = carreras_comunes.copy()
distribucion_modificada['Otras carreras'] = otros_total

# Mapeo de nombres personalizados
etiquetas_personalizadas = {
    'Licenciatura en Ciencias de la Computación': 'Computación',
    'Licenciatura en Ciencias Biológicas': 'Biología',
    'Licenciatura en Ciencias Físicas': 'Física',
    'Licenciatura en Ciencias de Datos': 'Datos',
    'Licenciatura en Ciencias Químicas': 'Química',
    'Licenciatura en Ciencias Matemáticas': 'Matemática',
    'Licenciatura en Ciencias de la Atmósfera': 'Atmósfera',
    'Licenciatura en Ciencia y Tecnología de Alimentos': 'Alimentos',
    'Licenciatura en Ciencias Geológicas': 'Geología',
    'Otras carreras': 'Otras carreras'
}

# Renombrar los índices según el diccionario
distribucion_modificada.index = distribucion_modificada.index.map(lambda x: etiquetas_personalizadas.get(x, x))

# Ordenar por cantidad descendente
distribucion_modificada = distribucion_modificada.sort_values(ascending=False)

# Graficar
distribucion_modificada.plot(kind='bar', figsize=(9, 5), color='royalblue')
plt.title('Distribución de carrera principal', fontsize=15)
plt.xlabel('Carrera Principal', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.tight_layout()
plt.show()
